In [35]:
import pandas as pd
import numpy as np

np.random.seed(0)

def generate_item_df(num_items = 20, weight_range = (1, 10), value_range = (1, 5)):
    weights = np.random.uniform(*weight_range, num_items)
    values = np.random.uniform(*value_range, num_items)
    return pd.DataFrame({
        'Item': [f'item {i}' for i in range(num_items)],
        'Weight': weights,
        'Value': values
    })

df = generate_item_df()
print(df.head(4))

     Item    Weight     Value
0  item 0  5.939322  4.914473
1  item 1  7.436704  4.196634
2  item 2  6.424870  2.845917
3  item 3  5.903949  4.122117


In [36]:
import random

def genetic_algorithm(df):
    weights = df["Weight"].tolist()
    values = df["Value"].tolist()
    n = len(values)

    max_weight = 30

    population_size = 100
    num_generations = 50
    mutation_rate = 0.01

    population = [[random.randint(0, 1) for _ in range(n)] for _ in range(population_size)]

    def fitness(individual):
        total_value = sum(value if individual[i] else 0 for i, value in enumerate(values))
        total_weight = sum(weight if individual[i] else 0 for i, weight in enumerate(weights))
        if total_weight > max_weight:
            return 0
        return total_value

    def tournament_selection(population, k=3):
        return max(random.sample(population, k), key=fitness)

    def crossover(parent1, parent2):
        crossover_point = random.randint(1, len(parent1) - 1)
        return parent1[:crossover_point] + parent2[crossover_point:], parent2[:crossover_point] + parent1[crossover_point:]

    def mutate(individual):
        for i in range(len(individual)):
            if random.random() < mutation_rate:
                individual[i] = 1 - individual[i] # Flip bit
        return individual

    for generation in range(num_generations):
        new_population = []
        for _ in range(population_size // 2):
            parent1 = tournament_selection(population)
            parent2 = tournament_selection(population)
            offspring1, offspring2 = crossover(parent1, parent2)
            new_population.append(mutate(offspring1))
            new_population.append(mutate(offspring2))
        population = new_population

    return max(population, key=fitness)

for _ in range(5):
    items = [df.iloc[i] for i, bit in enumerate(genetic_algorithm(df)) if bit]
    print(f"Items: {[item['Item'] for item in items]}")
    print(f"Total weight: {sum([item['Weight'] for item in items])}")
    print(f"Total value: {sum([item['Value'] for item in items])}")
    print()


Items: ['item 0', 'item 3', 'item 11', 'item 15', 'item 16', 'item 18']
Total weight: 28.572864492919585
Total value: 24.827442025065434

Items: ['item 0', 'item 1', 'item 7', 'item 11', 'item 15']
Total weight: 29.94620081482642
Total value: 21.457260040036658

Items: ['item 0', 'item 1', 'item 3', 'item 11', 'item 14', 'item 15', 'item 16']
Total weight: 29.645482555503687
Total value: 25.324243169619255

Items: ['item 0', 'item 3', 'item 7', 'item 9', 'item 14', 'item 15', 'item 16']
Total weight: 29.92565465684649
Total value: 24.467997583175922

Items: ['item 0', 'item 3', 'item 11', 'item 15', 'item 16', 'item 18']
Total weight: 28.572864492919585
Total value: 24.827442025065434

